# Chapter 2: A/B Testing: Evaluating a Change to the System 

In [ ]:
import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300

In [ ]:
# Note that Python converts between booleans and floats like this:
print (float(True), float(False))
print (bool(0), bool(1))

In [ ]:
clr1 = "#333333"
clr2 = "#777777"
clr3 = "#AAAAAA"

def save_fig(fig_num):
    pass
    # fig_dir = "/Users/dsweet2/Desktop/Tuning Up/Chapter 2/"
    #for ext in ["eps", "png"]:
    #    plt.savefig(f"{fig_dir}/CH02_FIG_{fig_num}_sweet.{ext}")

In [ ]:
def horizonal_line(y0):
    c = plt.axis()
    plt.plot([c[0], c[1]], [y0, y0], '--', linewidth=1, color=clr3);

## 2.1	Randomize to Isolate the change being tested

In [ ]:
def cost(strategy_A, server_1):
    return 10 + float(strategy_A) - 2*float(server_1)

In [ ]:
cost(strategy_A=True, server_1=True)

In [ ]:
def biased_experiment():
    cost_A = cost(strategy_A=True, server_1=True)
    cost_B = cost(strategy_A=False, server_1=False)

    return cost_B - cost_A

In [ ]:
biased_experiment()

In [ ]:
def unbiased_experiment():
    cost_A_1 = cost(strategy_A=True, server_1=True)
    cost_A_2 = cost(strategy_A=True, server_1=False)
    cost_B_1 = cost(strategy_A=False, server_1=False)
    cost_B_2 = cost(strategy_A=False, server_1=True)    
    cost_A = (cost_A_1 + cost_A_2)/2
    cost_B = (cost_B_1 + cost_B_2)/2
    
    return cost_B - cost_A

In [ ]:
unbiased_experiment()

In [ ]:
def randomized_experiment():
    cost_A = cost(strategy_A=True, server_1=bool(np.random.randint(2)))
    cost_B = cost(strategy_A=False, server_1=bool(np.random.randint(2)))
            
    return cost_B - cost_A

In [ ]:
np.random.seed(17)
print (randomized_experiment())
print (randomized_experiment())
print (randomized_experiment())

In [ ]:
def calculate_expectation():
    data = []
    for _ in range(1000):
        data.append(randomized_experiment())
    data = np.array(data) 
    return data.mean()

In [ ]:
np.random.seed(17); calculate_expectation()

In [ ]:
np.random.choice([-1,1], size=(10,))

In [ ]:
def cost_complex(strategy_A, nuisance_factors):
    return float(strategy_A) + nuisance_factors.sum()/20

def randomized_experiment_complex(num_nuisance_factors):
    nuisance_factors_A = np.random.choice([-1,1],
                                          size=(num_nuisance_factors,))
    cost_A = cost_complex(True, nuisance_factors_A)
    nuisance_factors_B = np.random.choice([-1,1],
                                          size=(num_nuisance_factors,))
    cost_B = cost_complex(False, nuisance_factors_B)
            
    return cost_B - cost_A

In [ ]:
np.random.seed(17)
print (randomized_experiment_complex(num_nuisance_factors=100))
print (randomized_experiment_complex(num_nuisance_factors=100))
print (randomized_experiment_complex(num_nuisance_factors=100))

In [ ]:
def calculate_expectation_complex(num_nuisance_factors):
    data = [randomized_experiment_complex(num_nuisance_factors)
            for _ in range(1000)]
    return np.array(data).mean()

In [ ]:
np.random.seed(17);
print (calculate_expectation_complex(num_nuisance_factors=100))

## 2.2	Replicate and determine the number of measurements to take

### 2.2.1	Randomization induces variability

In [ ]:
np.random.seed(17);
data_rec_10000 = np.array([
    randomized_experiment_complex(num_nuisance_factors=100)
                           for _ in range(10000)])

In [ ]:
print (data_rec_10000.mean() - 2*data_rec_10000.std())
print (data_rec_10000.mean() + 2*data_rec_10000.std())

In [ ]:
plt.hist(data_rec_10000, 15, color=clr1);
plt.xlabel(r'$cost_B - cost_A$')
plt.ylabel('count');
save_fig(6)

In [ ]:
def aggregate_measurement(num_measurements):
    measurements = [randomized_experiment_complex(
                                num_nuisance_factors=100)
                    for _ in range(num_measurements)]
    return np.array(measurements).mean()

In [ ]:
np.random.seed(17);
aggregate_measurement(10)

In [ ]:
def bootstrap_mean(data, num_measurements):
    # Compute means by resampling from data rather than generating new data.
    # This is done here just to save the time that would be required to
    #  generate new data on each call to this function.
    i = np.random.randint(data.shape[0], size=(num_measurements,))
    return data[i].mean()

In [ ]:
np.random.seed(17);
data_10 = np.array([bootstrap_mean(data_rec_10000, 10) for _ in range(10000)])

In [ ]:
plt.hist(data_rec_10000,15,color="#333333");
plt.hist(data_10,15,color="#777777");
plt.xlabel(r'$cost_B - cost_A$')
plt.ylabel('count');
plt.legend(['single measurement', 'average of\n10 measurements'], fontsize=8);
save_fig(7)

In [ ]:
np.random.seed(17);
data_100 = np.array([bootstrap_mean(data_rec_10000, 100) for _ in range(10000)])

In [ ]:
plt.hist(data_rec_10000,15,color="#333333");
plt.hist(data_10,15,color="#777777");
plt.hist(data_100,15,color="#BBBBBB");
plt.xlabel(r'$cost_B - cost_A$')
plt.ylabel('count');
plt.legend(['single measurement', 'average of\n10 measurements',
           'average of\n100 measurements'], fontsize=8);
save_fig(8)

### 2.2.2 Quantify variability with standard error

In [ ]:
def calc_SD1():
    measurements = [randomized_experiment_complex(num_nuisance_factors=100)
                        for _ in range(100)]
    return np.array(measurements).std()

np.random.seed(17); calc_SD1()

In [ ]:
def se_vs_N(data_rec_10000, expectation, hline=None, histogram=True, max_N=10000):
    np.random.seed(17);
    N = np.arange(1, max_N)
    if histogram:
        data = []
        for n in N:
            for _ in range(10):
                m = bootstrap_mean(data_rec_10000, n)
                data.append( (n,m) )
        data = np.array(data)
        plt.plot(data[:,0], expectation + data[:,1] + 1, '.', markersize=1, color="#AAAAAA")
        
    sd = data_rec_10000.std()
    if max_N <= 100:
        fmt = ".--"
    else:
        fmt = "--"
        
    plt.plot(N, expectation + sd/np.sqrt(N), fmt, color="#333333")
    plt.plot(N, expectation - sd/np.sqrt(N), fmt, color="#333333")
    plt.xlabel('number of measurements, N')

    if histogram:
        plt.ylabel('$cost_B - cost_A$')
        plt.legend(['aggreagate (average of N) measurements', 'standard error (S.E.)'], fontsize=8,
                  loc = 'lower right');
    else:
        plt.ylabel('SE of $cost_B - cost_A$')

    if hline is not None:
        horizonal_line(hline)


In [ ]:
se_vs_N(data_rec_10000, expectation=-1)
save_fig(9)

### 2.2.3 Minimize measurement costs

In [ ]:
se_vs_N(data_rec_10000, expectation=-1, hline=0)
save_fig(10)

In [ ]:
PS = .3
se_vs_N(data_rec_10000, expectation=-PS, hline=0, histogram=False)
save_fig(11)

In [ ]:
se_vs_N(data_rec_10000, expectation=-PS, hline=0, histogram=False,
        max_N=10)
save_fig(12)

In [ ]:
def aggregate_measurement_model(num_measurements):
    PS = .3
    SD1 = .707
    measurements = -PS + SD1*np.random.normal(size=(num_measurements,))
    return measurements.mean()

In [ ]:
plt.hist(np.random.normal(size=(10000,)), 25, color="#333333");
plt.hist(4 + .5*np.random.normal(size=(10000,)), 25, color="#777777");
c = plt.axis()
plt.axis([c[0], c[1], 0, 1700])
plt.legend(['np.random.normal(size=(10000,))', '4 + .5*np.random.normal(size=(10000,))'], loc='upper left');
save_fig(13)

In [ ]:
np.random.seed(17)
aggregate_measurement_model(num_measurements=6)

In [ ]:
def probability_false_negative(N):
    samples  = np.array([aggregate_measurement_model(num_measurements=N) for _ in range(10000)])
    return len(np.where(samples > 0)[0]) / len(samples)

In [ ]:
np.random.seed(17)
probability_false_negative(N=6)

In [ ]:
def aggregate_measurement_model_t(num_measurements, num_samples_stddev):
    PS = .3
    SD1 = .707
    measurements = -PS + SD1*np.random.standard_t(df=num_samples_stddev-1, size=(num_measurements,))
    return measurements.mean()

In [ ]:
def probability_false_negative_t(N, num_samples_stddev):
    samples  = np.array([aggregate_measurement_model_t(num_measurements=N,
                                                      num_samples_stddev=num_samples_stddev) for _ in range(10000)])
    return len(np.where(samples>0)[0]) / len(samples)

In [ ]:
np.random.seed(17)
probability_false_negative_t(N=6, num_samples_stddev=10)

In [ ]:
def overlap_fn_fp():
    SD1 = .707
    PS = .3
    data = []
    thresh = None
    best_N = None
    for N in range(1,100):
        SE = SD1 / np.sqrt(N)
        upper_fp_alpha = 0 + 1.96*SE
        lower_fp_alpha = 0 - 1.96*SE
        upper_fn_beta = -PS + .84*SE
        lower_fn_beta = -PS - .84*SE
        if thresh is None and upper_fn_beta <= lower_fp_alpha:
            thresh = (upper_fn_beta + lower_fp_alpha) / 2
            best_N = N
        data.append( (N, upper_fp_alpha, lower_fp_alpha, upper_fn_beta, lower_fn_beta) )
    return np.array(data), thresh, best_N

In [ ]:
data, thresh, best_N = overlap_fn_fp()

In [ ]:
plt.fill_between(data[:,0], data[:,3], data[:,4], color=clr3, alpha=.75, linewidth=1)


plt.xlabel('number of measurements, N')
plt.ylabel('$cost_B - cost_A$')
plt.legend([r'Reject false negatives with $\beta = .20$'])
save_fig(14)

In [ ]:
plt.fill_between(data[:,0], data[:,1], data[:,2], color=clr2, alpha=.75, linewidth=1)


plt.xlabel('number of measurements, N')
plt.ylabel('$cost_B - cost_A$')
plt.legend([r'Reject false positives with $\alpha = .05$'])

save_fig(15)

In [ ]:
print (thresh, best_N)

In [ ]:
data, thresh, best_N = overlap_fn_fp()
plt.fill_between(data[:,0], data[:,1], data[:,2], color=clr2, alpha=.75, linewidth=1)
plt.fill_between(data[:,0], data[:,3], data[:,4], color=clr3, alpha=.75, linewidth=1)

circle = mpl.patches.Ellipse( (best_N, thresh), .2*25, .2, color=clr1, fill=False)#, transform=plt.gca().transAxes)
plt.gcf().gca().add_artist(circle)

plt.xlabel('number of measurements, N')
plt.ylabel('$cost_B - cost_A$')
plt.legend([r'$\alpha = .05$', r'$\beta = .20$'], fontsize=8,
          loc = 'lower right');

save_fig(16)

In [ ]:
probability_false_negative_t(20, 10)

## 2.3	Run and analyze the A/B test

In [ ]:
# Table 2.1
alpha = .05
for N_small in [10, 30, 100, 300, 1000]:
    k = scipy.stats.t.ppf(1-alpha, df=N_small)
    print (N_small, k)

## 2.4	Early stopping produces invalid conclusions

In [ ]:
def t_stat_vs_sample():
    measurements = np.array([])
    t_stat = []
    threshold = []
    alpha=.05
    for df in range(1, 100):
        measurements = np.append(measurements, np.random.normal())
        if df > 1:
            mu = measurements[:df].mean()
            sd = measurements[:df].std()
            t = np.sqrt(df) * mu / sd
        else:
            t = np.nan
        t_stat.append(t)
        threshold.append(scipy.stats.t.ppf(1-alpha, df=df))
    t_stat = np.array(t_stat)
    threshold = np.array(threshold)
    return t_stat, threshold

In [ ]:
seed = 70366 #np.random.randint(100000)
np.random.seed(seed)

t_stat, threshold = t_stat_vs_sample()

plt.plot(threshold, '--k', color=clr1)
plt.plot(t_stat, color=clr2, linewidth=1);
plt.plot(-threshold, '--k', color=clr1)
plt.xlabel('sample')
plt.legend(['threshold, k', 't statistic'])

save_fig(18)

In [ ]:
def false_positive_rates():
    num_ab_tests = 100
    fp_at_end = 0
    fp_early_stopping = 0
    for _ in range(num_ab_tests):
        t_stat, threshold = t_stat_vs_sample() 
        if t_stat[-1] > threshold[-1]:
            fp_at_end += 1
        i = np.where(t_stat[1:] > threshold[1:])[0]
        if len(i) > 0:
            fp_early_stopping += 1
    return fp_at_end / num_ab_tests, fp_early_stopping / num_ab_tests
        

In [ ]:
np.random.seed(17);false_positive_rates()

In [ ]:
# Faster verions of t_stat_vs_sample() and false_positive_rates() used to
#  generate data for Figure 2.19.  The original versions were easier to use for
#  teaching, but too slow to generate the figure in a reasonable amount of time.
def t_stat_vs_sample_fast(N):
    measurements = np.random.normal(size=(N-1,))
    N = np.arange(2, N+1)
    sx = np.cumsum(measurements)
    sxx = np.cumsum(measurements**2)
    mu = sx/N
    sd = np.sqrt(sxx/N - mu**2)
    t_stats = np.sqrt(N) * mu/sd
    return t_stats

def false_positive_rates_fast(N):
    num_ab_tests = 10000
    fp_at_end = 0
    fp_early_stopping = 0
    for _ in range(num_ab_tests):
        t_stat = t_stat_vs_sample_fast(N)    
        if t_stat[-1] > 1.65:
            fp_at_end += 1
        i = np.where(t_stat > 1.65)[0]
        if len(i) > 0:
            fp_early_stopping += 1
    return fp_at_end / num_ab_tests, fp_early_stopping / num_ab_tests
        

In [ ]:
np.random.seed(17)
false_positive_rates_fast(1000)

In [ ]:
np.random.seed(17)
fpr = []
for N in [10, 30, 100, 300, 1000, 3000, int(1e4), int(3e4), int(1e5), int(3e5), int(1e6)]:
    fp_N = false_positive_rates_fast(N)[1]
    print (N, fp_N)
    fpr.append( (N, fp_N)) 

In [ ]:
fpr = np.array(fpr)
plt.semilogx(fpr[:,0], fpr[:,1], '.--', color=clr1);
plt.xlabel('N')
plt.ylabel('false positive rate')
save_fig(19)